In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor  
import random
from matplotlib import pyplot as plt

In [ ]:
train_df=pd.read_csv('../input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Train.csv')

In [ ]:
test_df=pd.read_csv('../input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.isnull().values.any()

In [ ]:
train_df.shape

In [ ]:
train_df.duplicated().sum()

In [ ]:
train_df.drop_duplicates(inplace=True)
train_df['InvoiceDate']=pd.to_datetime(train_df['InvoiceDate'])
train_df['InvoiceDay']=train_df['InvoiceDate'].dt.day
train_df['InvoiceMonth']=train_df['InvoiceDate'].dt.month
train_df['InvoiceYear']=train_df['InvoiceDate'].dt.year
train_df['InvoiceHour']=train_df['InvoiceDate'].dt.hour

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
fig = plt.figure(figsize = (15,20))
ax = fig.gca()
train_df.hist(ax=ax)

In [ ]:
train_df['InvoiceNo'].value_counts(normalize=True)

In [ ]:
test_df['InvoiceNo'].value_counts(normalize=True)

In [ ]:
test_df[test_df['Quantity']<0]

In [ ]:
train_df['WeekDay']= train_df['InvoiceDate'].dt.dayofweek
train_df['WeekDay']=train_df['InvoiceNo'].apply(lambda x : 1 if x<5 else 0)
train_df.drop('InvoiceDate', inplace=True, axis=1)
train_df.drop('InvoiceNo', inplace=True, axis=1)
train_df.drop('InvoiceHour', inplace=True, axis=1)
train_df.drop('WeekDay', inplace=True, axis=1)
train_df.drop('InvoiceDay', inplace=True, axis=1)
train_df.drop('InvoiceYear', inplace=True, axis=1)

In [ ]:
train_df.corr(method='spearman')['UnitPrice']

In [ ]:
train_df.skew()

In [ ]:
target = train_df['UnitPrice']
train_df.drop('UnitPrice', inplace=True, axis=1)

In [ ]:
train_df.head()

In [ ]:
from category_encoders import LeaveOneOutEncoder
enc = LeaveOneOutEncoder(cols= ['Description', 'StockCode', 'CustomerID','Country'])
train_df = enc.fit_transform(train_df, target)

In [ ]:
train_df

In [ ]:
# x = train_df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# train_df = pd.DataFrame(x_scaled)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.pipeline import Pipeline
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', MinMaxScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', MinMaxScaler()),('LASSO', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', MinMaxScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', MinMaxScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', MinMaxScaler()),('CART', DecisionTreeRegressor())])))
pipelines.append(('ScaledGBM', Pipeline([('Scaler', MinMaxScaler()),('GBM', GradientBoostingRegressor())])))

results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=21)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# dtc = Lasso(alpha = 0.0).fit(X_train,y_train)

In [ ]:
reg=DecisionTreeRegressor(random_state=0)

In [ ]:
# param_grid = {"criterion": ["mse"], 
#               "max_features": [2,3,4,5,6,7,8], 
#               "min_samples_split": [5,10, 15, 20, 25, 30],
#               "max_depth": [2],
#               "min_samples_leaf": [20, 40, 60, 80, 100],
#               "max_leaf_nodes": [5]
#               }
# ## Comment in order to publish in kaggle.

# grid_cv = GridSearchCV(reg, param_grid, cv=5, n_jobs=6, verbose=5)
# grid_cv.fit(X_train, y_train)
# print("Tuned Logistic Regression Parameters: {}".format(grid_cv.best_params_))  
# print("Best score is {}".format(grid_cv.best_score_)) 
# # print(reg.score(X_test, y_test))

In [ ]:
dtc = DecisionTreeRegressor(criterion='mse', max_depth=2, max_leaf_nodes=5, min_samples_leaf=20, min_samples_split=5, random_state=0, max_features= 4)
dtc.fit(X_train, y_train)

In [ ]:
# GBR = GradientBoostingRegressor()
# parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
#                   'subsample'    : [0.9, 0.5, 0.2, 0.1],
#                   'n_estimators' : [100,500,1000, 1500],
#                   'max_depth'    : [4,6,8,10]
#                  }
# grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1, verbose=True)
# grid_GBR.fit(X_train, y_train)
# print(" Results from Grid Search " )
# print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
# print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
# print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
# xgb1 = XGBRegressor()
# parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
#               'objective':['reg:linear'],
#               'learning_rate': [.03, 0.05, .07], #so called `eta` value
#               'max_depth': [5, 6, 7],
#               'min_child_weight': [4],
#               'silent': [1],
#               'subsample': [0.7],
#               'colsample_bytree': [0.7],
#               'n_estimators': [500]}

# xgb_grid = GridSearchCV(xgb1, parameters, cv = 2, n_jobs = 5, verbose=True)
# xgb_grid.fit(X_train, y_train)
# print(xgb_grid.best_score_)
# print(xgb_grid.best_params_)

In [ ]:
y_predicted = dtc.predict(X_test)


In [ ]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test,y_predicted)))

In [ ]:
test_df['InvoiceDate']=pd.to_datetime(test_df['InvoiceDate'])
test_df['InvoiceDay']=test_df['InvoiceDate'].dt.day
test_df['InvoiceMonth']=test_df['InvoiceDate'].dt.month
test_df['InvoiceYear']=test_df['InvoiceDate'].dt.year
test_df['InvoiceHour']=test_df['InvoiceDate'].dt.hour
test_df['WeekDay']= test_df['InvoiceDate'].dt.dayofweek
test_df['WeekDay']=test_df['InvoiceNo'].apply(lambda x : 1 if x<5 else 0)
test_df.drop('InvoiceDate', inplace=True, axis=1)
test_df.drop('InvoiceNo', inplace=True, axis=1)
test_df.drop('InvoiceHour', inplace=True, axis=1)
test_df.drop('WeekDay', inplace=True, axis=1)
test_df.drop('InvoiceDay', inplace=True, axis=1)
test_df.drop('InvoiceYear', inplace=True, axis=1)

In [ ]:
test_df.head()

In [ ]:
# test_df = pd.DataFrame(enc.transform(test_df['Description', 'StockCode', 'CustomerID']))
test_df = enc.transform(test_df)

In [ ]:
test_df.head()

In [ ]:
# test_df = pd.DataFrame(min_max_scaler.transform(test_df.values))

In [ ]:
out = dtc.predict(test_df)

In [ ]:
sample_df=pd.read_csv('../input/the-great-indian-hiring-hackathon/Participants_Data_TGIH/Sample Submission.csv')

In [ ]:
np.unique(out, return_counts=True)

In [ ]:
sample_df['UnitPrice'] = out.round(2)

In [ ]:
sample_df

In [ ]:
sample_df.to_csv('my_submission_file34.csv', index=False)

In [ ]:
test_df.shape

In [ ]:
282159